In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re

In [3]:
# reading in csv file

csv = pd.read_csv('vi_03_2024.csv').reset_index()

In [4]:
# renaming variables

csv.columns = ['index', 'task_id', 'created_at', 'completed_at', 'last_modified', 'name', 'section', 'assignee', 'assignee_email', 'start_date', 'due_date', 'tags', 'notes', 'projects', 'parent_task', 'blocked_by', 'blocking', 'artwork', 'id_no', 'artist', 'interaction_level', 'date_of_interaction', 'date_found', 'form_submitter', 'interaction_witness', 'location', 'age_group', 'response_category', 'damaged', 'resolution_detail', 'type_of_touch', 'visitor_reaction', 'corrective_measures', 'interaction_tagging', 'av', 'date', 'cmoa_location', 'type_of_issue', 'new_recurring', 'nearby_artwork', 'activity_type', 'task_progress', 'date_reported', 'priority', 'created_by', 'issue_type']
csv = csv.drop('index', axis=1)
csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10821 entries, 0 to 10820
Data columns (total 45 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   task_id              10821 non-null  int64  
 1   created_at           10821 non-null  object 
 2   completed_at         10574 non-null  object 
 3   last_modified        10821 non-null  object 
 4   name                 10820 non-null  object 
 5   section              10819 non-null  object 
 6   assignee             8944 non-null   object 
 7   assignee_email       8944 non-null   object 
 8   start_date           0 non-null      float64
 9   due_date             10799 non-null  object 
 10  tags                 0 non-null      float64
 11  notes                10818 non-null  object 
 12  projects             10819 non-null  object 
 13  parent_task          2 non-null      object 
 14  blocked_by           0 non-null      float64
 15  blocking             0 non-null     

In [5]:
# cleaning

csv = csv.apply(lambda x: x.str.strip().str.lower() if x.dtype == 'object' else x)
csv['location'] = csv['location'].replace('scaife gallery one', 'scaife gallery 01')

csv['created_at'] = pd.to_datetime(csv['created_at'])
csv['completed_at'] = pd.to_datetime(csv['completed_at'])
csv['last_modified'] = pd.to_datetime(csv['last_modified'])
csv['start_date'] = pd.to_datetime(csv['start_date'])
csv['due_date'] = pd.to_datetime(csv['due_date'])

In [6]:
# pulls list of top artworks each month

def month_most(month, year):
    new_df = csv.loc[(csv.created_at.dt.month == month) & (csv.created_at.dt.year == year)].reset_index()
    new_df = new_df[['created_at', 'artwork', 'artist', 'location', 'task_id']].dropna()
    new_df = new_df.groupby(['artwork', 'artist', 'location'])['task_id'].size().reset_index()
    new_df = new_df.sort_values('task_id', ascending=False)
    return(new_df.head(10))

In [8]:
csv.location.unique()

array(['scaife gallery 17', 'scaife gallery c', 'heinz gallery a',
       'heinz gallery b', 'scaife gallery 16', 'scaife gallery 08',
       'heinz gallery c', 'scaife gallery 01', 'scaife gallery 11',
       'scaife gallery 02', 'scaife gallery 04', 'scaife gallery 05',
       'scaife gallery 06', 'scaife gallery 13', 'scaife gallery 14',
       'scaife gallery 07', 'scaife lounge', 'lobby / foyer', nan,
       'scaife gallery 15', 'scaife gallery 09',
       'hall of sculpture / hos balcony', 'scaife gallery b',
       'forum gallery', 'hall of architecture (hoa)',
       'miniatures gallery', 'hac gallery h5', 'hac gallery h3',
       'hac gallery h1', 'hac gallery h4', 'hac gallery h2',
       'scaife gallery a', 'bruce gallery 20', 'bruce gallery 19',
       'hac gallery h6', 'bruce gallery 18', 'scaife gallery 12',
       'hac gallery h7', 'heinz gallery d', 'scaife staircase',
       'grand staircase', 'scaife gallery 10',
       'theater / theater hallway', 'outdoor / museum g

In [9]:
# pulls list of top artworks each month, by gallery/galleries

def month_gallery(month, year, location):
    new_df = csv.loc[(csv.created_at.dt.month == month) & (csv.created_at.dt.year == year)].reset_index()
    new_df = new_df[new_df.location.str.contains(location, na=False)].reset_index()
    new_df = new_df[['created_at', 'artwork', 'artist', 'location', 'task_id']].dropna()
    new_df = new_df.groupby(['artwork', 'artist', 'location'])['task_id'].size().reset_index()
    new_df = new_df.sort_values('task_id', ascending=False)
    return(new_df.head(10))

In [10]:
csv.interaction_level.unique()

array(['level 0', 'level 1', 'level 2', 'level 3', nan], dtype=object)

In [9]:
month_gallery(1, 2023, 'heinz gallery c')

,artwork,artist,location,task_id
2,oyonïk (the calling),édgar calel,heinz gallery c,4
3,stock garden,soun-gui kim,heinz gallery c,2
0,oyonik,edgar calel,heinz gallery c,1
1,oyonïk,edgar calel,heinz gallery c,1
4,the oyonïk,edgar calel,heinz gallery c,1
5,“the calling”,edgar calel,heinz gallery c,1


In [18]:
# pulls list of top artworks each month, year, gallery, artwork, and level

def search(month, year, location, artwork="", level="", query=""):
    if month == None:
        new_df = csv.loc[(csv.created_at.dt.year == year)].reset_index()
    else:
        new_df = csv.loc[(csv.created_at.dt.month == month) & (csv.created_at.dt.year == year)].reset_index()
    #new_df = csv.loc[(csv.created_at.dt.month == month) & (csv.created_at.dt.year == year)].reset_index()
    new_df = new_df[new_df.location.str.contains(location, na=False)].reset_index()
    new_df = new_df[new_df.artwork.str.contains(artwork, na=False)].rename_axis('test1').reset_index()
    new_df = new_df[new_df.interaction_level.str.contains(level, na=False)].rename_axis('test2').reset_index()
    new_df = new_df[new_df.notes.str.contains(query, na=False)].rename_axis('test3').reset_index()
    new_df = new_df[['created_at', 'artwork', 'artist', 'location', 'interaction_level', 'notes', 'task_id']].dropna()
    new_df = new_df.groupby(['artwork', 'artist', 'location', 'interaction_level', 'notes'])['task_id'].size().reset_index()
    new_df = new_df.sort_values('task_id', ascending=False)
    return(new_df.head(25))

In [19]:
search(month=6, year=2023, location="scaife", artwork="", level="level 2", query="")

,artwork,artist,location,interaction_level,notes,task_id
0,23 years of night,mohammed sami,scaife gallery 16,level 2,submitter name:\nga zeja copes\n\nfound or obs...,1
97,synthesa,franz west,scaife gallery 16,level 2,submitter name:\ndonald daube\n\nfound or obse...,1
123,two panels: green orange,ellsworth kelly,scaife gallery 13,level 2,submitter name:\ngregory bedel \n\nfound or ob...,1
124,two panels: green orange,ellsworth kelly,scaife gallery 13,level 2,submitter name:\nkelsey hinshaw\n\nfound or ob...,1
125,untitled,aleksandra aleksandrovna exster,scaife gallery 09,level 2,submitter name:\ndonald daube\n\nfound or obse...,1
126,untitled,donald judd,scaife gallery 13,level 2,submitter name:\nava parker\n\nfound or observ...,1
127,untitled,donald judd,scaife gallery 13,level 2,submitter name:\ndonald daube\n\nfound or obse...,1
128,untitled,donald judd,scaife gallery 13,level 2,submitter name:\ndonald daube\n\nfound or obse...,1
129,untitled,donald judd,scaife gallery 13,level 2,submitter name:\ndonald daube\n\nfound or obse...,1
130,untitled,donald judd,scaife gallery 13,level 2,submitter name:\ngregory bedel\n\nfound or obs...,1


In [12]:
# pulls list of top artworks each month, by artwork

def month_artwork(month, year, artwork):
    new_df = csv.loc[(csv.created_at.dt.month == month) & (csv.created_at.dt.year == year)].reset_index()
    new_df = new_df[new_df.artwork.str.contains(artwork, na=False)].reset_index()
    new_df = new_df[['created_at', 'artwork', 'artist', 'location', 'interaction_level', 'task_id']].dropna()
    new_df = new_df.groupby(['artwork', 'artist', 'location', 'interaction_level'])['task_id'].size().reset_index()
    new_df = new_df.sort_values('task_id', ascending=False)
    return(new_df.head(10))

In [15]:
month_artwork(2, 2023, 'untitled' )

,artwork,artist,location,interaction_level,task_id
15,untitled: upturnedhouse,phyllida barlow,scaife gallery 16,level 2,5
13,untitled selection of arpilleras,anonymous chilean women,scaife gallery 02,level 2,4
8,untitled,tith kanitha,heinz gallery b,level 2,3
6,untitled,laura owens,scaife gallery 16,level 2,2
10,untitled (red butterfly over green),mark grotjahn,scaife gallery 16,level 2,2
11,untitled floor works,tith kanitha,heinz gallery b,level 2,2
0,all works untitled,tith kanitha,heinz gallery b,level 2,1
1,untitled,ali golastaneh,hac gallery h5,level 2,1
2,untitled,anonymous chilean women,scaife gallery 02,level 2,1
3,untitled,christopher wool,scaife gallery 16,level 2,1


In [16]:
def string_search(string):
    string_df = csv[csv.notes.str.contains(string, na=False)].reset_index()
    string_df = string_df[['artwork', 'artist', 'location', 'task_id']].dropna()
    string_df = string_df.groupby(['artwork', 'artist', 'location'])['task_id'].size().reset_index()
    string_df = string_df.sort_values('task_id', ascending=False)
    return(string_df.head(10))

In [17]:
string_search('selfie')

,artwork,artist,location,task_id
4,the toilet of venus,simon vouet,scaife gallery 04,2
6,two panels: green orange,ellsworth kelly,scaife gallery 13,2
0,by the river,edward redfield,scaife gallery 08,1
1,irrational rest (2),jessica vaughn,forum gallery,1
2,right?,banu cennetoğlu,hall of sculpture / hos balcony,1
3,string of puppies,jeff koons,heinz gallery a,1
5,this has no name ii,luis felipe noé,scaife gallery 02,1
7,untitled,donald judd,scaife gallery 13,1
8,untitledno. 9,agnes martin,scaife gallery 13,1
9,vignette (wishing well),kerry james marshall,forum gallery,1
